In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm


In [2]:
import pandas as pd
file_path = '/home/abrar/Desktop/Code/Temporal HPC/normalized_data.csv'

# Read CSV and ensure datetime columns are parsed correctly
time_columns = ['submit_time', 'eligible_time', 'start_time', 'end_time', 'wait_time']
df = pd.read_csv(file_path, parse_dates=time_columns)
# Now 'df' contains the data from the second sheet

/tmp/ipykernel_1989129/84532322.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file_path, parse_dates=time_columns)


In [57]:
# Temporal distribution analysis
# Convert submit_time to hour of day and day of week
df['hour_of_day'] = df['submit_time'].dt.hour
df['day_of_week'] = df['submit_time'].dt.day_name()
df['month'] = df['submit_time'].dt.month

# Get distributions
print("Hour of day distribution:")
print(df['hour_of_day'].value_counts().sort_index())
print("\nDay of week distribution:")
print(df['day_of_week'].value_counts())
print("\nMonthly distribution:")
print(df['month'].value_counts().sort_index())

Hour of day distribution:
hour_of_day
0      4680
1      3880
2      3645
3      3653
4      3834
5      4312
6      6420
7     11364
8     12706
9     11611
10    10179
11    11690
12     8799
13    13013
14    14091
15     9886
16     8403
17     8840
18     4967
19     7375
20     6316
21     6719
22     7563
23     7369
Name: count, dtype: int64

Day of week distribution:
day_of_week
Tuesday      34127
Wednesday    33466
Friday       32642
Thursday     31801
Monday       24452
Saturday     21010
Sunday       13817
Name: count, dtype: int64

Monthly distribution:
month
5     19376
6     33923
7     33178
8     29692
9     33887
10    41259
Name: count, dtype: int64


In [58]:
# Job size and duration statistics
print("\nJob size statistics:")
print(df[['num_cores_alloc', 'num_nodes_alloc', 'num_gpus_alloc', 'mem_alloc', 'run_time']].describe())

# Power usage statistics
print("\nPower usage statistics:")
print(df[['mean_cpu_power', 'mean_mem_power', 'mean_node_power']].describe())


Job size statistics:
       num_cores_alloc  num_nodes_alloc  num_gpus_alloc      mem_alloc  \
count    191315.000000    191315.000000   191315.000000  191315.000000   
mean        107.315872         1.173761        4.127314     213.540993   
std         106.953631         0.732607        3.212431     190.284860   
min           4.000000         1.000000        0.000000       0.000000   
25%          32.000000         1.000000        4.000000     118.000000   
50%         128.000000         1.000000        4.000000     237.000000   
75%         128.000000         1.000000        4.000000     237.000000   
max       20736.000000       162.000000      648.000000   38475.000000   

            run_time  
count  191315.000000  
mean     5767.922732  
std      3778.020112  
min         1.000000  
25%      2858.890321  
50%      5581.545522  
75%      8422.589206  
max     23330.168557  

Power usage statistics:
       mean_cpu_power  mean_mem_power  mean_node_power
count   191315.000000   

In [59]:
# Calculate correlations between numerical columns
corr_cols = ['cores_per_task', 'num_tasks', 'num_cores_alloc', 'num_nodes_alloc', 
             'num_gpus_alloc', 'mem_alloc', 'mean_cpu_power', 'mean_mem_power', 
             'mean_node_power', 'run_time']
correlation_matrix = df[corr_cols].corr()
print("\nCorrelation matrix:")
print(correlation_matrix)


Correlation matrix:
                 cores_per_task  num_tasks  num_cores_alloc  num_nodes_alloc  \
cores_per_task         1.000000  -0.259773         0.268369         0.048834   
num_tasks             -0.259773   1.000000         0.147112         0.238280   
num_cores_alloc        0.268369   0.147112         1.000000         0.838456   
num_nodes_alloc        0.048834   0.238280         0.838456         1.000000   
num_gpus_alloc         0.072371   0.128644         0.818108         0.883935   
mem_alloc              0.189469   0.145238         0.903711         0.862834   
mean_cpu_power         0.055080   0.346748         0.639818         0.672793   
mean_mem_power         0.049872   0.281220         0.781548         0.923168   
mean_node_power        0.657201   0.011314         0.669077         0.402031   
run_time               0.610215   0.028178         0.670590         0.436848   

                 num_gpus_alloc  mem_alloc  mean_cpu_power  mean_mem_power  \
cores_per_task      

In [60]:

# For hour groups
df['hour_group'] = pd.cut(df['hour_of_day'], 
                         bins=[0, 6, 12, 18, 24], 
                         labels=['night', 'morning', 'afternoon', 'evening'])

# For core allocation, use custom bins based on the distribution we saw
df['size_group'] = pd.cut(df['num_cores_alloc'], 
                         bins=[0, 32, 64, 128, 256, float('inf')],
                         labels=['very_small', 'small', 'medium', 'large', 'very_large'])

# For runtime, use custom bins based on the distribution
df['runtime_group'] = pd.cut(df['run_time'],
                            bins=[0, 2000, 4000, 6000, 8000, float('inf')],
                            labels=['very_short', 'short', 'medium', 'long', 'very_long'])

# Create combined stratification feature
df['strata'] = df['hour_group'].astype(str) + '_' + \
               df['size_group'].astype(str) + '_' + \
               df['runtime_group'].astype(str)

# Check strata distribution
strata_counts = df['strata'].value_counts()
print("Number of strata:", len(strata_counts))
print("\nSample sizes per stratum:")
print("Min:", strata_counts.min())
print("Max:", strata_counts.max())
print("Mean:", strata_counts.mean())
print("Median:", strata_counts.median())

Number of strata: 90

Sample sizes per stratum:
Min: 1
Max: 14242
Mean: 2125.722222222222
Median: 325.0


Processing

In [61]:
# First, let's remove tiny strata that could cause issues
# Keep only strata with at least 20 samples to ensure meaningful splits
valid_strata = strata_counts[strata_counts >= 20].index
df_filtered = df[df['strata'].isin(valid_strata)].copy()

# Check how many samples we retained
print("Original samples:", len(df))
print("Filtered samples:", len(df_filtered))
print("Percentage retained:", (len(df_filtered)/len(df))*100, "%")

# Check new strata distribution
new_strata_counts = df_filtered['strata'].value_counts()
print("\nNew strata statistics:")
print("Number of strata:", len(new_strata_counts))
print("Min samples per stratum:", new_strata_counts.min())
print("Max samples per stratum:", new_strata_counts.max())

Original samples: 191315
Filtered samples: 191207
Percentage retained: 99.94354859786216 %

New strata statistics:
Number of strata: 78
Min samples per stratum: 20
Max samples per stratum: 14242


First Split for GNN Training (Power Prediction)

In [62]:
from sklearn.model_selection import train_test_split

# First split: separate out the final test set (10%)
train_val_idx, test_idx = train_test_split(
    df_filtered.index,
    test_size=0.1,
    stratify=df_filtered['strata'],
    random_state=42
)

# Then split remaining data into train (80% of total) and validation (10% of total)
train_idx, val_idx = train_test_split(
    train_val_idx,
    test_size=0.111,  # 0.111 of 90% is 10% of total
    stratify=df_filtered.loc[train_val_idx, 'strata'],
    random_state=42
)

# Create the datasets
gnn_train_data = df_filtered.loc[train_idx]
gnn_val_data = df_filtered.loc[val_idx]
gnn_test_data = df_filtered.loc[test_idx]

# Print split sizes
print("GNN Training set size:", len(gnn_train_data))
print("GNN Validation set size:", len(gnn_val_data))
print("GNN Test set size:", len(gnn_test_data))

# Verify distribution preservation
for split_name, split_data in [("Train", gnn_train_data), 
                              ("Val", gnn_val_data), 
                              ("Test", gnn_test_data)]:
    print(f"\n{split_name} set statistics:")
    print(f"Mean CPU power: {split_data['mean_cpu_power'].mean():.2f}")
    print(f"Mean node power: {split_data['mean_node_power'].mean():.2f}")
    print(f"Average cores: {split_data['num_cores_alloc'].mean():.2f}")

GNN Training set size: 152984
GNN Validation set size: 19102
GNN Test set size: 19121

Train set statistics:
Mean CPU power: 142.37
Mean node power: 840.20
Average cores: 107.41

Val set statistics:
Mean CPU power: 141.78
Mean node power: 840.45
Average cores: 107.05

Test set statistics:
Mean CPU power: 142.78
Mean node power: 839.84
Average cores: 106.99


In [63]:
# Sort data by submit_time within each split
gnn_train_data = gnn_train_data.sort_values('submit_time')
gnn_val_data = gnn_val_data.sort_values('submit_time')
gnn_test_data = gnn_test_data.sort_values('submit_time')

# Create temporal sequences for RL
# Let's first analyze the time spans
for split_name, split_data in [("Train", gnn_train_data), 
                              ("Val", gnn_val_data), 
                              ("Test", gnn_test_data)]:
    time_span = split_data['submit_time'].max() - split_data['submit_time'].min()
    jobs_per_day = len(split_data) / (time_span.total_seconds() / (24*3600))
    print(f"\n{split_name} set:")
    print(f"Time span: {time_span.days} days")
    print(f"Average jobs per day: {jobs_per_day:.2f}")


Train set:
Time span: 160 days
Average jobs per day: 954.19

Val set:
Time span: 159 days
Average jobs per day: 119.65

Test set:
Time span: 159 days
Average jobs per day: 119.78


create the RL episodes. For HPC scheduling, we want episodes that:

- Are long enough to capture meaningful scheduling decisions
- Short enough to provide many training episodes
- Account for daily power price variations

In [64]:
import numpy as np
# Create episodes of 24 hours each (to capture full day cycles)
def create_episodes(df, episode_duration_hours=24):
    start_time = df['submit_time'].min()
    end_time = df['submit_time'].max()
    
    episodes = []
    current_time = start_time
    
    while current_time < end_time:
        episode_end = current_time + pd.Timedelta(hours=episode_duration_hours)
        
        # Get jobs submitted during this episode
        episode_jobs = df[
            (df['submit_time'] >= current_time) & 
            (df['submit_time'] < episode_end)
        ].copy()
        
        if len(episode_jobs) > 0:
            episodes.append(episode_jobs)
            
        current_time = episode_end
    
    return episodes

# Create episodes for each split
train_episodes = create_episodes(gnn_train_data)
val_episodes = create_episodes(gnn_val_data)
test_episodes = create_episodes(gnn_test_data)

# Print episode statistics
for split_name, split_episodes in [("Train", train_episodes), 
                                 ("Val", val_episodes), 
                                 ("Test", test_episodes)]:
    num_episodes = len(split_episodes)
    avg_jobs = np.mean([len(ep) for ep in split_episodes])
    min_jobs = np.min([len(ep) for ep in split_episodes])
    max_jobs = np.max([len(ep) for ep in split_episodes])
    
    print(f"\n{split_name} episodes:")
    print(f"Number of episodes: {num_episodes}")
    print(f"Average jobs per episode: {avg_jobs:.2f}")
    print(f"Min jobs per episode: {min_jobs}")
    print(f"Max jobs per episode: {max_jobs}")


Train episodes:
Number of episodes: 155
Average jobs per episode: 986.99
Min jobs per episode: 9
Max jobs per episode: 7888

Val episodes:
Number of episodes: 154
Average jobs per episode: 124.04
Min jobs per episode: 1
Max jobs per episode: 1119

Test episodes:
Number of episodes: 154
Average jobs per episode: 124.16
Min jobs per episode: 1
Max jobs per episode: 1115


GNN training structure

In [65]:
# Features for GNN (selected based on correlations with mean_node_power)
gnn_features = [
    'cores_per_task',     # Strong correlation (0.657)
    'num_cores_alloc',    # Strong correlation (0.669)
    'num_nodes_alloc',    # Important for node-level prediction
    'num_gpus_alloc',     # Relevant for power consumption
    'mem_alloc',          # Moderate correlation (0.580)
    'run_time'           # Strongest correlation (0.962)
]

# Single target variable
power_target = ['mean_node_power']

# Prepare the data structure for GNN
def prepare_gnn_data(df, features=gnn_features, target=power_target):
    """
    Prepare data for GNN training with KNN structure
    """
    # Normalize features (important for KNN)
    scaler = StandardScaler()
    X = scaler.fit_transform(df[features])
    y = df[target].values
    
    # For each job, we'll find k similar jobs
    k = 5  # Can be tuned
    knn = NearestNeighbors(n_neighbors=k+1)  # +1 because it includes the point itself
    knn.fit(X)
    
    # Get indices of k nearest neighbors for each job
    distances, indices = knn.kneighbors(X)
    
    return X, y, indices[:, 1:], scaler  # Exclude self from neighbors

print("Preparing GNN data splits:")
for split_name, split_data in [("Train", gnn_train_data), 
                              ("Val", gnn_val_data), 
                              ("Test", gnn_test_data)]:
    print(f"\n{split_name} set:")
    print(f"Number of jobs: {len(split_data)}")
    print(f"Mean node power: {split_data['mean_node_power'].mean():.2f}")
    print(f"Std node power: {split_data['mean_node_power'].std():.2f}")

Preparing GNN data splits:

Train set:
Number of jobs: 152984
Mean node power: 840.20
Std node power: 182.99

Val set:
Number of jobs: 19102
Mean node power: 840.45
Std node power: 183.30

Test set:
Number of jobs: 19121
Mean node power: 839.84
Std node power: 184.18


In [66]:
# Features for GNN (selected based on correlations with mean_node_power)
gnn_features = [
    'cores_per_task',     # Strong correlation (0.657)
    'num_cores_alloc',    # Strong correlation (0.669)
    'num_nodes_alloc',    # Important for node-level prediction
    'num_gpus_alloc',     # Relevant for power consumption
    'mem_alloc',          # Moderate correlation (0.580)
    'run_time'           # Strongest correlation (0.962)
]

# Single target variable
power_target = ['mean_node_power']

# Prepare the data structure for GNN
def prepare_gnn_data(df, features=gnn_features, target=power_target):
    """
    Prepare data for GNN training with KNN structure
    """
    # Normalize features (important for KNN)
    scaler = StandardScaler()
    X = scaler.fit_transform(df[features])
    y = df[target].values
    
    # For each job, we'll find k similar jobs
    k = 5  # Can be tuned
    knn = NearestNeighbors(n_neighbors=k+1)  # +1 because it includes the point itself
    knn.fit(X)
    
    # Get indices of k nearest neighbors for each job
    distances, indices = knn.kneighbors(X)
    
    return X, y, indices[:, 1:], scaler  # Exclude self from neighbors

print("Preparing GNN data splits:")
for split_name, split_data in [("Train", gnn_train_data), 
                              ("Val", gnn_val_data), 
                              ("Test", gnn_test_data)]:
    print(f"\n{split_name} set:")
    print(f"Number of jobs: {len(split_data)}")
    print(f"Mean node power: {split_data['mean_node_power'].mean():.2f}")
    print(f"Std node power: {split_data['mean_node_power'].std():.2f}")

Preparing GNN data splits:

Train set:
Number of jobs: 152984
Mean node power: 840.20
Std node power: 182.99

Val set:
Number of jobs: 19102
Mean node power: 840.45
Std node power: 183.30

Test set:
Number of jobs: 19121
Mean node power: 839.84
Std node power: 184.18


problem

In [67]:
class PowerPredictionGNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim=128):
        super(PowerPredictionGNN, self).__init__()
        
        # Wider network
        self.node_embed = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU()
        )
        
        # Deeper graph convolutions
        self.conv1 = GCNConv(hidden_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.conv3 = GCNConv(hidden_dim, hidden_dim)
        self.bn3 = nn.BatchNorm1d(hidden_dim)
        
        # More robust prediction head
        self.power_pred = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim//2),
            nn.ReLU(),
            nn.Linear(hidden_dim//2, 1)
        )
        
    def forward(self, x, edge_index):
        # Initial embeddings
        x = self.node_embed(x)
        
        # Graph convolutions with residual connections
        x1 = F.relu(self.bn1(self.conv1(x, edge_index)))
        x2 = F.relu(self.bn2(self.conv2(x1, edge_index))) + x1
        x3 = F.relu(self.bn3(self.conv3(x2, edge_index))) + x2
        
        # Power prediction
        return self.power_pred(x3)
    
    
def create_graph_batch(features, neighbor_indices, targets, batch_size=1024):
    """
    Create graph batches ensuring consistent shapes between features and targets
    """
    graphs = []
    num_samples = len(features)
    
    for i in range(0, num_samples, batch_size):
        batch_end = min(i + batch_size, num_samples)
        batch_features = features[i:batch_end]
        batch_neighbors = neighbor_indices[i:batch_end]
        batch_targets = targets[i:batch_end]
        
        # Create edges based on KNN within this batch
        edge_index = []
        for job_idx, neighbors in enumerate(batch_neighbors):
            # Only add edges for neighbors that are within this batch
            valid_neighbors = [n - i for n in neighbors if i <= n < batch_end]
            for neighbor in valid_neighbors:
                edge_index.append([job_idx, neighbor])
                edge_index.append([neighbor, job_idx])
        
        if not edge_index:  # If no valid edges in this batch
            continue
            
        edge_index = torch.tensor(edge_index).t().contiguous()
        
        # Create graph with matching features and targets
        graph = Data(
            x=torch.FloatTensor(batch_features),
            edge_index=edge_index,
            y=torch.FloatTensor(batch_targets).unsqueeze(-1)  # Add dimension for consistency
        )
        graphs.append(graph)
    
    return graphs

# Modified training configuration
config = {
    'hidden_dim': 128,
    'learning_rate': 0.001,
    'batch_size': 1024,
    'epochs': 100,
    'early_stopping_patience': 15
}

# Print some statistics about the data
print(f"Total number of jobs in training: {len(gnn_train_data)}")
print(f"Total number of jobs in validation: {len(gnn_val_data)}")
print(f"Total number of jobs in test: {len(gnn_test_data)}")

# Initialize model and start training
input_dim = len(gnn_features)
model = PowerPredictionGNN(input_dim=input_dim)


print("\nPreparing data...")
train_graphs, val_graphs, test_graphs, scaler,  *_ = prepare_training_data()

# Print graph statistics
print("\nGraph statistics:")
print(f"Number of training graphs: {len(train_graphs)}")
print(f"Number of validation graphs: {len(val_graphs)}")
print(f"Number of test graphs: {len(test_graphs)}")

print("\nSample graph details:")
if train_graphs:
    g = train_graphs[0]
    print(f"First training graph - Nodes: {g.x.size(0)}, Edges: {g.edge_index.size(1)//2}")

print("\nStarting training...")
train_losses, val_losses = train_gnn(model, train_graphs, val_graphs, config)

Total number of jobs in training: 152984
Total number of jobs in validation: 19102
Total number of jobs in test: 19121

Preparing data...
Calculating neighbors for training set...
Calculating neighbors for validation set...
Calculating neighbors for test set...


ValueError: not enough values to unpack (expected 7, got 5)

In [51]:
def power_prediction_loss(pred_power, true_power):
    """
    Custom loss function ensuring shape consistency
    """
    # Ensure shapes match
    pred_power = pred_power.squeeze()
    true_power = true_power.squeeze()
    
    # Check shapes are identical
    assert pred_power.shape == true_power.shape, f"Shape mismatch: pred={pred_power.shape}, true={true_power.shape}"
    
    # Compute normalized MSE
    pred_norm = (pred_power - pred_power.mean()) / (pred_power.std() + 1e-8)
    true_norm = (true_power - true_power.mean()) / (true_power.std() + 1e-8)
    
    mse_loss = F.mse_loss(pred_norm, true_norm)
    
    # Compute relative error
    relative_error = torch.abs(pred_power - true_power) / (torch.abs(true_power) + 1e-8)
    relative_loss = torch.mean(relative_error)
    
    return mse_loss + 0.1 * relative_loss

In [52]:
def prepare_rl_state(episode_df, gnn_model, scaler):
    """
    Prepare RL state with GNN power predictions
    """
    # Normalize features
    features_normalized = scaler.transform(episode_df[gnn_features])
    
    # Get power predictions from GNN
    with torch.no_grad():
        power_pred = gnn_model(features_normalized)
    
    # Create state dictionary
    state = {
        'job_features': episode_df[gnn_features].values,
        'predicted_power': power_pred.numpy(),
        'submit_time': episode_df['submit_time'].values,
        'time_of_day': episode_df['submit_time'].dt.hour.values
    }
    
    return state

In [53]:
class EarlyStopping:
    def __init__(self, patience=7, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        
    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

def train_gnn(model, train_graphs, val_graphs, config):
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
    early_stopping = EarlyStopping(patience=config['early_stopping_patience'])
    
    train_losses = []
    val_losses = []
    
    for epoch in range(config['epochs']):
        # Training
        model.train()
        train_loss = 0
        for graph in train_graphs:
            optimizer.zero_grad()
            out = model(graph.x, graph.edge_index)
            loss = power_prediction_loss(out, graph.y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for graph in val_graphs:
                out = model(graph.x, graph.edge_index)
                val_loss += power_prediction_loss(out, graph.y).item()
        
        train_loss /= len(train_graphs)
        val_loss /= len(val_graphs)
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        print(f'Epoch {epoch+1:03d}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
        
        # Early stopping
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break
            
    return train_losses, val_losses

# Evaluation metrics
def evaluate_predictions(model, test_graphs, power_scaler):
    model.eval()
    predictions = []
    true_values = []
    
    with torch.no_grad():
        for graph in test_graphs:
            pred = model(graph.x, graph.edge_index)
            # Ensure 2D shape before inverse transform
            pred = pred.squeeze().numpy().reshape(-1, 1)
            true = graph.y.squeeze().numpy().reshape(-1, 1)
            
            # Inverse transform
            pred = power_scaler.inverse_transform(pred)
            true = power_scaler.inverse_transform(true)
            
            predictions.extend(pred.flatten())
            true_values.extend(true.flatten())
    
    predictions = np.array(predictions)
    true_values = np.array(true_values)
    
    # Calculate metrics
    mse = mean_squared_error(true_values, predictions)
    mae = mean_absolute_error(true_values, predictions)
    r2 = r2_score(true_values, predictions)
    
    results = {
        'mse': mse,
        'rmse': np.sqrt(mse),
        'mae': mae,
        'r2': r2,
        'mean_relative_error': np.mean(np.abs(predictions - true_values) / (true_values + 1e-8)),
        'mean_pred': np.mean(predictions),
        'mean_true': np.mean(true_values),
        'std_pred': np.std(predictions),
        'std_true': np.std(true_values)
    }
    
    # Add some percentile errors
    for p in [25, 50, 75, 90]:
        error_percentile = np.percentile(np.abs(predictions - true_values), p)
        results[f'error_p{p}'] = error_percentile
    
    return results

In [54]:


def calculate_neighbors(features, k=5):
    """Calculate k-nearest neighbors for each job"""
    knn = NearestNeighbors(n_neighbors=k+1)  # +1 because it includes the point itself
    knn.fit(features)
    distances, indices = knn.kneighbors(features)
    return indices[:, 1:]  # Exclude self from neighbors

def prepare_training_data():
    """
    Prepare training data with proper scaling
    """
    # Normalize features
    feature_scaler = StandardScaler()
    train_features = feature_scaler.fit_transform(gnn_train_data[gnn_features])
    val_features = feature_scaler.transform(gnn_val_data[gnn_features])
    test_features = feature_scaler.transform(gnn_test_data[gnn_features])
    
    # Scale power values
    power_scaler = StandardScaler()
    train_target = power_scaler.fit_transform(gnn_train_data[['mean_node_power']])
    val_target = power_scaler.transform(gnn_val_data[['mean_node_power']])
    test_target = power_scaler.transform(gnn_test_data[['mean_node_power']])
    
    # Calculate neighbor indices
    print("Calculating neighbors for training set...")
    train_neighbors = calculate_neighbors(train_features)
    print("Calculating neighbors for validation set...")
    val_neighbors = calculate_neighbors(val_features)
    print("Calculating neighbors for test set...")
    test_neighbors = calculate_neighbors(test_features)
    
    # Create graph batches
    train_graphs = create_graph_batch(train_features, train_neighbors, train_target)
    val_graphs = create_graph_batch(val_features, val_neighbors, val_target)
    test_graphs = create_graph_batch(test_features, test_neighbors, test_target)
    
    return train_graphs, val_graphs, test_graphs, feature_scaler, power_scaler


# Initialize model
input_dim = len(gnn_features)  # Number of features
model = PowerPredictionGNN(input_dim=input_dim)

# Print model architecture
print("Model architecture:")
print(model)

# Initialize and train
print("\nPreparing data...")
# Fix the unpacking to match the 5 return values
train_graphs, val_graphs, test_graphs, feature_scaler, power_scaler = prepare_training_data()

print("\nModel architecture:")
# Initialize model with the new architecture
model = PowerPredictionGNN(input_dim=len(gnn_features), hidden_dim=config['hidden_dim'])
print(model)

print("\nStarting training...")
train_losses, val_losses = train_gnn(model, train_graphs, val_graphs, config)

print("\nEvaluating model...")
# Pass both scalers to evaluation
test_metrics = evaluate_predictions(model, test_graphs, power_scaler)
for metric, value in test_metrics.items():
    print(f"{metric}: {value:.4f}")

Model architecture:
PowerPredictionGNN(
  (node_embed): Sequential(
    (0): Linear(in_features=6, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=128, bias=True)
    (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
  )
  (conv1): GCNConv(128, 128)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): GCNConv(128, 128)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): GCNConv(128, 128)
  (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (power_pred): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
  

Working GNN + RL Code to prompt

In [3]:
# GNN

# --- Data Loading and Preprocessing ---

file_path = '/home/abrar/Desktop/Code/Temporal HPC/normalized_data.csv'

# Load data
time_columns = ['submit_time', 'eligible_time', 'start_time', 'end_time', 'wait_time']
df = pd.read_csv(file_path, parse_dates=time_columns)

# Feature Engineering (Simplified for GNN Focus)
df['hour_of_day'] = df['submit_time'].dt.hour
df['day_of_week'] = df['submit_time'].dt.day_name()
df['month'] = df['submit_time'].dt.month

# --- Data Analysis (Keep relevant parts) ---

# print("Hour of day distribution:")
# print(df['hour_of_day'].value_counts().sort_index())
# print("\nDay of week distribution:")
# print(df['day_of_week'].value_counts())
# print("\nMonthly distribution:")
# print(df['month'].value_counts().sort_index())

# print("\nJob size statistics:")
# print(df[['num_cores_alloc', 'num_nodes_alloc', 'num_gpus_alloc', 'mem_alloc', 'run_time']].describe())

# print("\nPower usage statistics:")
# print(df[['mean_cpu_power', 'mean_mem_power', 'mean_node_power']].describe())

corr_cols = ['cores_per_task', 'num_tasks', 'num_cores_alloc', 'num_nodes_alloc',
             'num_gpus_alloc', 'mem_alloc', 'mean_cpu_power', 'mean_mem_power',
             'mean_node_power', 'run_time']
correlation_matrix = df[corr_cols].corr()
# print("\nCorrelation matrix:")
# print(correlation_matrix)

# --- Stratified Splitting (Keep, but simplified) ---

# Create simplified stratification based on core allocation and runtime
df['size_group'] = pd.cut(df['num_cores_alloc'],
                         bins=[0, 32, 64, 128, 256, float('inf')],
                         labels=['very_small', 'small', 'medium', 'large', 'very_large'])
df['runtime_group'] = pd.cut(df['run_time'],
                            bins=[0, 2000, 4000, 6000, 8000, float('inf')],
                            labels=['very_short', 'short', 'medium', 'long', 'very_long'])
df['strata'] = df['size_group'].astype(str) + '_' + df['runtime_group'].astype(str)

# Remove small strata
valid_strata = df['strata'].value_counts()[df['strata'].value_counts() >= 20].index
df_filtered = df[df['strata'].isin(valid_strata)].copy()

# Split data (stratified)
train_val_idx, test_idx = train_test_split(
    df_filtered.index, test_size=0.1, stratify=df_filtered['strata'], random_state=42
)
train_idx, val_idx = train_test_split(
    train_val_idx, test_size=0.111, stratify=df_filtered.loc[train_val_idx, 'strata'], random_state=42
)

gnn_train_data = df_filtered.loc[train_idx]
gnn_val_data = df_filtered.loc[val_idx]
gnn_test_data = df_filtered.loc[test_idx]

# --- GNN Data Preparation ---

# Features and Target (Clearly Defined)
gnn_features = [
    'cores_per_task', 'num_cores_alloc', 'num_nodes_alloc',
    'num_gpus_alloc', 'mem_alloc', 'shared', 'priority',
    'num_tasks',
]


power_target = ['mean_node_power']

def prepare_gnn_data(df, features, target, feature_scaler=None, power_scaler=None):
    """
    Prepares data for GNN, including feature scaling and neighbor calculation.
    """
    # Fit scalers only on training data, transform others
    if feature_scaler is None:
        feature_scaler = StandardScaler()
        X = feature_scaler.fit_transform(df[features])
    else:
        X = feature_scaler.transform(df[features])

    if power_scaler is None:
        power_scaler = StandardScaler()
        y = power_scaler.fit_transform(df[target])
    else:
        y = power_scaler.transform(df[target])

    # KNN calculation
    knn = NearestNeighbors(n_neighbors=6)  # k=5 + 1 (itself)
    knn.fit(X)
    distances, indices = knn.kneighbors(X)
    neighbor_indices = indices[:, 1:]

    return X, y, neighbor_indices, feature_scaler, power_scaler

def create_graph_batch(features, neighbor_indices, targets, batch_size=1024):
    """
    Creates graph batches with edges based on KNN.
    """
    graphs = []
    num_samples = len(features)
    
    for i in range(0, num_samples, batch_size):
        batch_end = min(i + batch_size, num_samples)
        batch_features = features[i:batch_end]
        batch_neighbors = neighbor_indices[i:batch_end]
        batch_targets = targets[i:batch_end]
        
        # Create edges based on KNN within this batch
        edge_index = []
        
        for job_idx, neighbors in enumerate(batch_neighbors):
            for neighbor_idx in neighbors:
                # Convert global indices to local batch indices
                local_job_idx = job_idx
                local_neighbor_idx = neighbor_idx - i
        
                # Only add edges for neighbors that are within this batch
                if 0 <= local_neighbor_idx < len(batch_features):
                    edge_index.append([local_job_idx, local_neighbor_idx])
                    edge_index.append([local_neighbor_idx, local_job_idx])
        
        if not edge_index:  # If no valid edges in this batch
            continue
            
        edge_index = torch.tensor(edge_index).t().contiguous()
        
        # Create graph with matching features and targets
        graph = Data(
            x=torch.FloatTensor(batch_features),
            edge_index=edge_index,
            y=torch.FloatTensor(batch_targets)  # Add dimension for consistency
        )
        graphs.append(graph)
        
    return graphs

# Prepare data (fit scalers on train, transform all)
train_X, train_y, train_neighbors, feature_scaler, power_scaler = prepare_gnn_data(gnn_train_data, gnn_features, power_target)
val_X, val_y, val_neighbors, _, _ = prepare_gnn_data(gnn_val_data, gnn_features, power_target, feature_scaler, power_scaler)
test_X, test_y, test_neighbors, _, _ = prepare_gnn_data(gnn_test_data, gnn_features, power_target, feature_scaler, power_scaler)

# Create graph batches
train_graphs = create_graph_batch(train_X, train_neighbors, train_y)
val_graphs = create_graph_batch(val_X, val_neighbors, val_y)
test_graphs = create_graph_batch(test_X, test_neighbors, test_y)

# --- GNN Model ---

class PowerPredictionGNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim=128):
        super(PowerPredictionGNN, self).__init__()

        self.node_embed = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU()
        )

        self.conv1 = GCNConv(hidden_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.conv3 = GCNConv(hidden_dim, hidden_dim)
        self.bn3 = nn.BatchNorm1d(hidden_dim)

        self.power_pred = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim // 2, 1)
        )

    def forward(self, x, edge_index):
        x = self.node_embed(x)
        x1 = F.relu(self.bn1(self.conv1(x, edge_index)))
        x2 = F.relu(self.bn2(self.conv2(x1, edge_index))) + x1
        x3 = F.relu(self.bn3(self.conv3(x2, edge_index))) + x2
        return self.power_pred(x3)

# --- Loss Function and Training ---

def power_prediction_loss(pred_power, true_power):
    """
    Custom loss function: Normalized MSE + Relative Error
    """
    pred_power = pred_power.squeeze()
    true_power = true_power.squeeze()

    pred_norm = (pred_power - pred_power.mean()) / (pred_power.std() + 1e-8)
    true_norm = (true_power - true_power.mean()) / (true_power.std() + 1e-8)

    mse_loss = F.mse_loss(pred_norm, true_norm)
    relative_error = torch.abs(pred_power - true_power) / (torch.abs(true_power) + 1e-8)
    relative_loss = torch.mean(relative_error)

    return mse_loss + 0.5 * relative_loss

class EarlyStopping:
    def __init__(self, patience=15, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

def train_gnn(model, train_graphs, val_graphs, config):
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
    early_stopping = EarlyStopping(patience=config['early_stopping_patience'])

    train_losses = []
    val_losses = []

    for epoch in range(config['epochs']):
        model.train()
        train_loss = 0
        for graph in train_graphs:
            optimizer.zero_grad()
            out = model(graph.x, graph.edge_index)
            loss = power_prediction_loss(out, graph.y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for graph in val_graphs:
                out = model(graph.x, graph.edge_index)
                val_loss += power_prediction_loss(out, graph.y).item()

        train_loss /= len(train_graphs)
        val_loss /= len(val_graphs)
        train_losses.append(train_loss)
        val_losses.append(val_loss)

        print(f'Epoch {epoch+1:03d}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

        early_stopping(val_loss)
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break

    return train_losses, val_losses

# --- Evaluation ---

def evaluate_predictions(model, test_graphs, power_scaler):
    model.eval()
    predictions = []
    true_values = []

    with torch.no_grad():
        for graph in test_graphs:
            pred = model(graph.x, graph.edge_index)
            pred = pred.squeeze().numpy().reshape(-1, 1)
            true = graph.y.squeeze().numpy().reshape(-1, 1)
            pred = power_scaler.inverse_transform(pred)
            true = power_scaler.inverse_transform(true)
            predictions.extend(pred.flatten())
            true_values.extend(true.flatten())

    predictions = np.array(predictions)
    true_values = np.array(true_values)

    mse = mean_squared_error(true_values, predictions)
    mae = mean_absolute_error(true_values, predictions)
    r2 = r2_score(true_values, predictions)

    results = {
        'mse': mse,
        'rmse': np.sqrt(mse),
        'mae': mae,
        'r2': r2,
        'mean_relative_error': np.mean(np.abs(predictions - true_values) / (true_values + 1e-8)),
        'mean_pred': np.mean(predictions),
        'mean_true': np.mean(true_values),
        'std_pred': np.std(predictions),
        'std_true': np.std(true_values)
    }

    for p in [25, 50, 75, 90]:
        error_percentile = np.percentile(np.abs(predictions - true_values), p)
        results[f'error_p{p}'] = error_percentile

    return results, predictions, true_values

# --- Training Configuration and Initialization ---

config = {
    'hidden_dim': 128,
    'learning_rate': 0.001,
    'batch_size': 1024,
    'epochs': 100,
    'early_stopping_patience': 15
}

input_dim = len(gnn_features)
model = PowerPredictionGNN(input_dim=input_dim, hidden_dim=config['hidden_dim'])
print(model)

# --- Training and Evaluation ---

print("Starting training...")
train_losses, val_losses = train_gnn(model, train_graphs, val_graphs, config)

print("Evaluating model...")
test_metrics, predictions, true_values = evaluate_predictions(model, test_graphs, power_scaler)
for metric, value in test_metrics.items():
    print(f"{metric}: {value:.4f}")


#RL

class HPCEnvironment:
    def __init__(self, gnn_model, feature_scaler, power_scaler, num_nodes=100):
        self.gnn_model = gnn_model
        self.feature_scaler = feature_scaler
        self.power_scaler = power_scaler
        self.num_nodes = num_nodes
        self.peak_hour_cost = 2.5
        self.off_peak_cost = 0.6
        self.power_history = []
        self.peak_threshold = None
        self.waiting_jobs = []
        self.completed_jobs = []
        self.running_jobs = []
        self.reset()

    def reset(self):
        """Reset environment state"""
        self.node_status = [True] * self.num_nodes
        self.current_time = pd.Timestamp.now().replace(hour=0, minute=0)
        self.running_jobs = []
        self.energy_consumption = 0
        self.peak_consumption = 0
        self.off_peak_consumption = 0
        self.total_cost = 0
        self.waiting_jobs = []
        self.completed_jobs = []
        return self._get_state()

    def _get_state(self):
        """Enhanced state representation"""
        is_peak_hour = 1.0 if 8 <= self.current_time.hour < 20 else 0.0
        hours_until_change = min(
            abs(8 - self.current_time.hour) if self.current_time.hour < 8 else
            abs(20 - self.current_time.hour) if 8 <= self.current_time.hour < 20 else
            abs(32 - self.current_time.hour),
            12
        ) / 12.0
        
        return np.array([
            sum(self.node_status) / self.num_nodes,  # Available nodes
            len(self.running_jobs) / self.num_nodes,  # System load
            self.current_time.hour / 24.0,  # Time of day
            is_peak_hour,  # Peak hour indicator
            hours_until_change,  # Hours until price change
            self.peak_consumption / (self.energy_consumption + 1e-8),  # Peak ratio
            len(self.waiting_jobs) / 50,  # Waiting job pressure
            self.total_cost / 100000  # Normalized total cost
        ])
    def _can_schedule(self, job):
        """Check if job can be scheduled"""
        return sum(self.node_status) >= job['num_nodes_alloc']

    def _allocate_nodes(self, job):
        """Allocate nodes to a job"""
        nodes_needed = job['num_nodes_alloc']
        allocated_nodes = []
        
        for i, available in enumerate(self.node_status):
            if available and len(allocated_nodes) < nodes_needed:
                allocated_nodes.append(i)
                self.node_status[i] = False
        
        return allocated_nodes

    def _update_running_jobs(self):
        """Update status of running jobs"""
        remaining_jobs = []
        for job, nodes, end_time in self.running_jobs:
            if end_time <= self.current_time:
                for node in nodes:
                    self.node_status[node] = True
                self.completed_jobs.append(job)
            else:
                remaining_jobs.append((job, nodes, end_time))
        self.running_jobs = remaining_jobs

    def _predict_power(self, job):
        """Predict power consumption for a job"""
        job_features = np.array([[job[f] for f in gnn_features]])
        scaled_features = self.feature_scaler.transform(job_features)
        x = torch.FloatTensor(scaled_features)
        edge_index = torch.tensor([[0], [0]], dtype=torch.long)
        
        self.gnn_model.eval()
        with torch.no_grad():
            pred = self.gnn_model(x, edge_index)
            pred = pred.squeeze().numpy().reshape(-1, 1)
            pred = self.power_scaler.inverse_transform(pred)
        return pred.item()

    def step(self, action, job):
        reward = 0
        done = False
        info = {'scheduled': False}
        
        is_peak_hour = 8 <= self.current_time.hour < 20
        power_price = self.peak_hour_cost if is_peak_hour else self.off_peak_cost
        
        if action == 0:  # Schedule now
            if self._can_schedule(job):
                power = job['mean_node_power']
                duration_hours = job['run_time'] / 3600
                
                # Calculate costs
                energy_cost = power * duration_hours * power_price
                min_possible_cost = power * duration_hours * self.off_peak_cost
                
                # Much stronger penalties for peak hour scheduling
                if is_peak_hour:
                    base_penalty = -5.0  # Increased base penalty
                    power_penalty = -2.0 * (power / 1000)  # Additional penalty for high-power jobs
                    reward = base_penalty + power_penalty
                else:
                    reward = 2.0  # Bonus for off-peak scheduling
                
                # Cost efficiency reward
                cost_efficiency = (min_possible_cost - energy_cost) / min_possible_cost
                reward += 3.0 * cost_efficiency
                
                # Update metrics
                self.energy_consumption += power * duration_hours
                self.total_cost += energy_cost
                if is_peak_hour:
                    self.peak_consumption += power * duration_hours
                
                info = {
                    'scheduled': True,
                    'energy_cost': energy_cost,
                    'is_peak': is_peak_hour,
                    'power': power
                }
            else:
                reward = -3.0
        else:  # Wait
            if is_peak_hour:
                reward = 1.0  # Reward for waiting during peak
                if job['mean_node_power'] > 1000:  # Extra reward for waiting with high-power jobs
                    reward += 1.0
            else:
                reward = -1.0  # Penalty for waiting during off-peak
            
            self.current_time += pd.Timedelta(minutes=30)
        
        # Update running jobs
        self._update_running_jobs()
        
        # End of day check
        if self.current_time.hour >= 23:
            done = True
            # Final reward based on peak ratio
            peak_ratio = self.peak_consumption / (self.energy_consumption + 1e-8)
            if peak_ratio < 0.3:
                reward += 10.0
            elif peak_ratio > 0.6:
                reward -= 10.0
        
        next_state = self._get_state()
        return next_state, reward, done, info

    
    def _predict_power(self, job):
        job_features = np.array([[job[f] for f in gnn_features]])
        scaled_features = self.feature_scaler.transform(job_features)
        x = torch.FloatTensor(scaled_features)
        edge_index = torch.tensor([[0], [0]], dtype=torch.long)
        
        self.gnn_model.eval()
        with torch.no_grad():
            pred = self.gnn_model(x, edge_index)
            pred = pred.squeeze().numpy().reshape(-1, 1)
            pred = self.power_scaler.inverse_transform(pred)
        return pred.item()
    
    def _can_schedule(self, job):
        return sum(self.node_status) >= job['num_nodes_alloc']
    
    def _allocate_nodes(self, job):
        nodes_needed = job['num_nodes_alloc']
        allocated_nodes = []
        
        for i, available in enumerate(self.node_status):
            if available and len(allocated_nodes) < nodes_needed:
                allocated_nodes.append(i)
                self.node_status[i] = False
                
        return allocated_nodes
    
    def _update_running_jobs(self):
        # Remove completed jobs and free their nodes
        remaining_jobs = []
        for job, nodes, end_time in self.running_jobs:
            if end_time <= self.current_time:
                for node in nodes:
                    self.node_status[node] = True
            else:
                remaining_jobs.append((job, nodes, end_time))
        self.running_jobs = remaining_jobs
    
    def _get_power_price(self):
        hour = self.current_time.hour
        if 8 <= hour < 20:  # Peak hours
            return 1.5
        return 1.0
    
    def get_expected_cost_difference(self, job):
        """Calculate cost difference between peak and off-peak scheduling"""
        power = job['mean_node_power']
        duration = job['run_time'] / 3600
        peak_cost = power * duration * self.peak_hour_cost
        off_peak_cost = power * duration * self.off_peak_cost
        return peak_cost - off_peak_cost

    def get_waiting_penalty(self, job):
        """Calculate penalty for waiting based on job characteristics"""
        waiting_jobs_count = len(self.waiting_jobs)
        system_load = len(self.running_jobs) / self.num_nodes
        urgency = min(job['run_time'] / 3600, 24)  # Cap at 24 hours
        
        base_penalty = -0.1 * waiting_jobs_count
        load_factor = -0.2 if system_load < 0.3 else 0.0  # Penalize waiting when system is underutilized
        urgency_factor = -0.1 * urgency / 24  # Higher penalty for long jobs
        
        return base_penalty + load_factor + urgency_factor
    

    class PPOMemory:
    def __init__(self):
        self.states = []
        self.actions = []
        self.probs = []
        self.vals = []
        self.rewards = []
        self.dones = []
    
    def generate_batches(self, batch_size):
        n_states = len(self.states)
        batch_start = np.arange(0, n_states, batch_size)
        indices = np.arange(n_states, dtype=np.int64)
        np.random.shuffle(indices)
        batches = [indices[i:i+batch_size] for i in batch_start]
        return np.array(self.states),\
               np.array(self.actions),\
               np.array(self.probs),\
               np.array(self.vals),\
               np.array(self.rewards),\
               np.array(self.dones),\
               batches
    
    def store_memory(self, state, action, probs, vals, reward, done):
        self.states.append(state)
        self.actions.append(action)
        self.probs.append(probs)
        self.vals.append(vals)
        self.rewards.append(reward)
        self.dones.append(done)
    
    def clear_memory(self):
        self.states = []
        self.actions = []
        self.probs = []
        self.vals = []
        self.rewards = []
        self.dones = []

class ActorNetwork(nn.Module):
    def __init__(self, input_dims, n_actions=3, hidden_dim=256):
        super(ActorNetwork, self).__init__()
        self.actor = nn.Sequential(
            nn.Linear(input_dims, hidden_dim),
            nn.ReLU(),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.LayerNorm(hidden_dim),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, n_actions),
            nn.Softmax(dim=-1)
        )
    
    def forward(self, state):
        return self.actor(state)

class CriticNetwork(nn.Module):
    def __init__(self, input_dims, hidden_dim=256):
        super(CriticNetwork, self).__init__()
        
        self.critic = nn.Sequential(
            nn.Linear(input_dims, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
        
    def forward(self, state):
        value = self.critic(state)
        return value
    

# 3. Add experience replay buffer
class ReplayBuffer:
    def __init__(self, capacity=10000):
        self.capacity = capacity
        self.buffer = []
        self.position = 0
    
    def push(self, state, action, reward, next_state, done):
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = (state, action, reward, next_state, done)
        self.position = (self.position + 1) % self.capacity
    
    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        state, action, reward, next_state, done = map(np.stack, zip(*batch))
        return state, action, reward, next_state, done


class TrainingMetrics:
    def __init__(self):
        self.peak_schedules = 0
        self.off_peak_schedules = 0
        self.total_wait_time = 0
        self.energy_costs = []
        self.peak_ratios = []
        
    def update(self, time, action, energy_cost=0, is_peak=False):
        if action == 0:  # Schedule
            if is_peak:
                self.peak_schedules += 1
            else:
                self.off_peak_schedules += 1
            self.energy_costs.append(energy_cost)
        else:  # Wait
            self.total_wait_time += 1 if action == 1 else 4
            
    def get_summary(self):
        return {
            'peak_schedule_ratio': self.peak_schedules / (self.peak_schedules + self.off_peak_schedules + 1e-8),
            'total_wait_hours': self.total_wait_time,
            'avg_energy_cost': np.mean(self.energy_costs) if self.energy_costs else 0
        }
    
    def train_scheduler(env, n_episodes=1000):
    input_dims = 8  # Updated state dimension for enhanced state space
    n_actions = 2  # Schedule now or wait
    
    actor = ActorNetwork(input_dims, n_actions)
    critic = CriticNetwork(input_dims)
    memory = PPOMemory()
    
    best_reward = float('-inf')
    
    # Prepare job queue once
    train_jobs = prepare_job_queue(gnn_train_data, n_jobs=100)
    
    for episode in range(n_episodes):
        state = env.reset()
        done = False
        total_reward = 0
        job_queue = train_jobs.copy()  # Reset job queue for each episode
        peak_schedules = 0
        total_schedules = 0
        
        while not done and len(job_queue) > 0:
            state_tensor = torch.FloatTensor(state)
            
            # Get action
            action_probs = actor(state_tensor)
            dist = Categorical(action_probs)
            action = dist.sample()
            
            # Get next job from queue
            current_job = job_queue[0]
            
            # Take action
            next_state, reward, done, info = env.step(action.item(), current_job)
            
            # Track peak vs off-peak scheduling
            if info['scheduled']:
                total_schedules += 1
                job_queue.pop(0)
                if info.get('is_peak', False):
                    peak_schedules += 1
            
            # Store transition
            memory.store_memory(state, action.item(), action_probs[action.item()].item(),
                              critic(state_tensor).item(), reward, done)
            
            state = next_state
            total_reward += reward
            
        # Update policy after episode
        if episode % 10 == 0:
            update_policy(memory, actor, critic)
            memory.clear_memory()
            
            # Print detailed metrics
            peak_ratio = peak_schedules / (total_schedules + 1e-8)
            print(f'Episode {episode}:')
            print(f'Total Reward: {total_reward:.2f}')
            print(f'Energy Cost: ${env.total_cost:.2f}')
            print(f'Energy Usage: {env.energy_consumption:.2f} kWh')
            print(f'Peak Schedule Ratio: {peak_ratio:.2%}')
            print(f'Average Cost per kWh: ${env.total_cost/env.energy_consumption:.2f}\n')
        
        # Save best model
        if total_reward > best_reward:
            best_reward = total_reward
            torch.save(actor.state_dict(), 'best_scheduler_actor.pth')
            torch.save(critic.state_dict(), 'best_scheduler_critic.pth')

    return actor, critic

# Create a job queue for testing
def prepare_job_queue(data, n_jobs=100):
    """Prepare a queue of jobs for testing"""
    jobs = data.sample(n=n_jobs).to_dict('records')
    # Sort by submit time to maintain temporal order
    jobs = sorted(jobs, key=lambda x: x['submit_time'])
    return jobs

# Add the ReplayBuffer class definition before the training loop
class ReplayBuffer:
    def __init__(self, capacity=10000):
        self.capacity = capacity
        self.buffer = []
        self.position = 0
    
    def push(self, state, action, reward, next_state, done):
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = (state, action, reward, next_state, done)
        self.position = (self.position + 1) % self.capacity
    
    def sample(self, batch_size):
        batch = random.sample(self.buffer, min(len(self.buffer), batch_size))
        state, action, reward, next_state, done = map(np.stack, zip(*batch))
        return state, action, reward, next_state, done

# Modify the training section
print("Starting experiment...")
env = HPCEnvironment(
    gnn_model=model,
    feature_scaler=feature_scaler,
    power_scaler=power_scaler,
    num_nodes=100
)

input_dims = 8  # New state dimension
n_actions = 2
actor = ActorNetwork(input_dims, n_actions)
critic = CriticNetwork(input_dims)

# Add optimizers and scheduler
actor_optimizer = torch.optim.Adam(actor.parameters(), lr=0.001)
critic_optimizer = torch.optim.Adam(critic.parameters(), lr=0.001)
actor_scheduler = ReduceLROnPlateau(actor_optimizer, mode='max', factor=0.5, patience=5, verbose=True)

# Create replay buffer
buffer = ReplayBuffer(capacity=10000)
batch_size = 64

train_jobs = prepare_job_queue(gnn_train_data, n_jobs=1000)
test_jobs = prepare_job_queue(gnn_test_data, n_jobs=100)

print("\nTraining scheduler...")
training_stats = []
best_reward = float('-inf')

for episode in range(100):
    state = env.reset()
    episode_rewards = []
    episode_actions = []
    episode_costs = []
    job_queue = train_jobs.copy()
    
    while len(job_queue) > 0:
        state_tensor = torch.FloatTensor(state)
        action_probs = actor(state_tensor)
        dist = Categorical(action_probs)
        action = dist.sample()
        
        current_job = job_queue[0]
        next_state, reward, done, info = env.step(action.item(), current_job)
        
        # Store experience in buffer
        buffer.push(state, action.item(), reward, next_state, done)
        
        # Update networks if we have enough samples
        if len(buffer.buffer) >= batch_size:
            states, actions, rewards, next_states, dones = buffer.sample(batch_size)
            
            # Convert to tensors
            states_tensor = torch.FloatTensor(states)
            actions_tensor = torch.LongTensor(actions)
            rewards_tensor = torch.FloatTensor(rewards)
            next_states_tensor = torch.FloatTensor(next_states)
            dones_tensor = torch.FloatTensor(dones)
            
            # Get values
            current_values = critic(states_tensor).squeeze()
            next_values = critic(next_states_tensor).squeeze().detach()
            
            # Calculate advantage
            advantages = rewards_tensor + (1 - dones_tensor) * 0.99 * next_values - current_values
            
            # Actor loss
            action_probs = actor(states_tensor)
            log_probs = torch.log(action_probs + 1e-10)
            actor_loss = -(advantages.detach() * log_probs[range(batch_size), actions_tensor]).mean()
            
            # Critic loss
            critic_loss = F.mse_loss(current_values, rewards_tensor + (1 - dones_tensor) * 0.99 * next_values)
            
            # Update networks
            actor_optimizer.zero_grad()
            actor_loss.backward()
            actor_optimizer.step()
            
            critic_optimizer.zero_grad()
            critic_loss.backward()
            critic_optimizer.step()
        
        if info['scheduled']:
            job_queue.pop(0)
        
        episode_rewards.append(reward)
        episode_actions.append(action.item())
        if 'energy_cost' in info:
            episode_costs.append(info['energy_cost'])
        
        state = next_state
        if done:
            break
    
    total_reward = sum(episode_rewards)
    schedule_rate = sum(1 for a in episode_actions if a == 0) / len(episode_actions)
    
    # Update learning rate scheduler
    actor_scheduler.step(total_reward)
    
    training_stats.append({
        'episode': episode,
        'total_reward': total_reward,
        'schedule_rate': schedule_rate,
        'energy_cost': env.total_cost,
        'energy_consumption': env.energy_consumption
    })
    
    if episode % 10 == 0:
        print(f"\nEpisode {episode}")
        print(f"Total Reward: {total_reward:.2f}")
        print(f"Schedule Rate: {schedule_rate:.2%}")
        print(f"Energy Cost: ${env.total_cost:.2f}")
        print(f"Energy Consumption: {env.energy_consumption:.2f} kWh")
    
    if total_reward > best_reward:
        best_reward = total_reward
        torch.save(actor.state_dict(), 'best_scheduler_actor.pth')

# Add optimizers and scheduler
actor_optimizer = torch.optim.Adam(actor.parameters(), lr=0.001)
critic_optimizer = torch.optim.Adam(critic.parameters(), lr=0.001)
actor_scheduler = ReduceLROnPlateau(actor_optimizer, mode='max', factor=0.5, patience=5) #, verbose=True

# Create replay buffer
buffer = ReplayBuffer(capacity=10000)
batch_size = 64

train_jobs = prepare_job_queue(gnn_train_data, n_jobs=1000)
test_jobs = prepare_job_queue(gnn_test_data, n_jobs=100)

print("\nTraining scheduler...")
training_stats = []
best_reward = float('-inf')

for episode in range(100):
    state = env.reset()
    episode_rewards = []
    episode_actions = []
    episode_costs = []
    job_queue = train_jobs.copy()
    
    while len(job_queue) > 0:
        state_tensor = torch.FloatTensor(state)
        action_probs = actor(state_tensor)
        dist = Categorical(action_probs)
        action = dist.sample()
        
        current_job = job_queue[0]
        next_state, reward, done, info = env.step(action.item(), current_job)
        
        # Store experience in buffer
        buffer.push(state, action.item(), reward, next_state, done)
        
        # Update networks if we have enough samples
        if len(buffer.buffer) >= batch_size:
            states, actions, rewards, next_states, dones = buffer.sample(batch_size)
            
            # Convert to tensors
            states_tensor = torch.FloatTensor(states)
            actions_tensor = torch.LongTensor(actions)
            rewards_tensor = torch.FloatTensor(rewards)
            next_states_tensor = torch.FloatTensor(next_states)
            dones_tensor = torch.FloatTensor(dones)
            
            # Get values
            current_values = critic(states_tensor).squeeze()
            next_values = critic(next_states_tensor).squeeze().detach()
            
            # Calculate advantage
            advantages = rewards_tensor + (1 - dones_tensor) * 0.99 * next_values - current_values
            
            # Actor loss
            action_probs = actor(states_tensor)
            log_probs = torch.log(action_probs + 1e-10)
            actor_loss = -(advantages.detach() * log_probs[range(batch_size), actions_tensor]).mean()
            
            # Critic loss
            critic_loss = F.mse_loss(current_values, rewards_tensor + (1 - dones_tensor) * 0.99 * next_values)
            
            # Update networks
            actor_optimizer.zero_grad()
            actor_loss.backward()
            actor_optimizer.step()
            
            critic_optimizer.zero_grad()
            critic_loss.backward()
            critic_optimizer.step()
        
        if info['scheduled']:
            job_queue.pop(0)
        
        episode_rewards.append(reward)
        episode_actions.append(action.item())
        if 'energy_cost' in info:
            episode_costs.append(info['energy_cost'])
        
        state = next_state
        if done:
            break
    
    total_reward = sum(episode_rewards)
    schedule_rate = sum(1 for a in episode_actions if a == 0) / len(episode_actions)
    
    # Update learning rate scheduler
    actor_scheduler.step(total_reward)
    
    training_stats.append({
        'episode': episode,
        'total_reward': total_reward,
        'schedule_rate': schedule_rate,
        'energy_cost': env.total_cost,
        'energy_consumption': env.energy_consumption
    })
    
    if episode % 10 == 0:
        print(f"\nEpisode {episode}")
        print(f"Total Reward: {total_reward:.2f}")
        print(f"Schedule Rate: {schedule_rate:.2%}")
        print(f"Energy Cost: ${env.total_cost:.2f}")
        print(f"Energy Consumption: {env.energy_consumption:.2f} kWh")
    
    if total_reward > best_reward:
        best_reward = total_reward
        torch.save(actor.state_dict(), 'best_scheduler_actor.pth')



class SchedulerAnalytics:
    def __init__(self):
        self.episode_metrics = []
        self.hourly_distributions = {}
        self.cost_metrics = []
        self.power_patterns = []
    
    def update(self, env, episode_num):
        """Update metrics after each episode"""
        # Peak vs Off-peak distribution
        peak_metrics = self.analyze_peak_distribution(env)
        
        # Cost efficiency
        cost_metrics = self.calculate_cost_efficiency(env)
        
        # Hourly job distribution
        hourly_dist = self.get_hourly_distribution(env)
        
        # Power consumption patterns
        power_metrics = self.analyze_power_patterns(env)
        
        metrics = {
            'episode': episode_num,
            'total_cost': env.total_cost,
            'energy_consumption': env.energy_consumption,
            **peak_metrics,
            **cost_metrics,
            'hourly_distribution': hourly_dist,
            **power_metrics
        }
        
        self.episode_metrics.append(metrics)
        return metrics
    
    def get_hourly_distribution(self, env):
        """Fixed version of hourly distribution calculation"""
        hourly_counts = {hour: 0 for hour in range(24)}
        for job in env.completed_jobs:
            hour = job['submit_time'].hour
            hourly_counts[hour] += 1
        return hourly_counts
    
    def analyze_peak_distribution(self, env):
        peak_hours = env.peak_consumption
        off_peak_hours = env.energy_consumption - env.peak_consumption
        peak_ratio = peak_hours / (env.energy_consumption + 1e-8)
        return {
            'peak_consumption': peak_hours,
            'off_peak_consumption': off_peak_hours,
            'peak_ratio': peak_ratio
        }
    
    def calculate_cost_efficiency(self, env):
        avg_cost_per_kwh = env.total_cost / (env.energy_consumption + 1e-8)
        potential_min_cost = env.energy_consumption * env.off_peak_cost
        cost_efficiency = potential_min_cost / (env.total_cost + 1e-8)
        return {
            'avg_cost_per_kwh': avg_cost_per_kwh,
            'cost_efficiency': cost_efficiency
        }
    
    def analyze_power_patterns(self, env):
        if not env.power_history:
            return {'avg_power': 0, 'high_power_ratio': 0}
        
        avg_power = np.mean(env.power_history)
        high_power_threshold = np.percentile(env.power_history, 75)
        high_power_jobs = sum(1 for p in env.power_history if p > high_power_threshold)
        high_power_ratio = high_power_jobs / len(env.power_history)
        
        return {
            'avg_power': avg_power,
            'high_power_ratio': high_power_ratio
        }

def run_experiment():
    print("Initializing environment...")
    env = HPCEnvironment(
        gnn_model=model,
        feature_scaler=feature_scaler,
        power_scaler=power_scaler,
        num_nodes=100
    )
    
    print("Creating networks...")
    input_dims = 8
    n_actions = 2
    actor = ActorNetwork(input_dims, n_actions)
    critic = CriticNetwork(input_dims)
    
    print("Creating analytics tracker...")
    analytics = SchedulerAnalytics()
    
    print("Preparing job queue...")
    # Start with a smaller job queue for testing
    train_jobs = prepare_job_queue(gnn_train_data, n_jobs=100)
    print(f"Job queue size: {len(train_jobs)}")
    
    print("\nStarting training...")
    best_reward = float('-inf')
    
    for episode in range(100):
        print(f"\nStarting Episode {episode}")
        state = env.reset()
        episode_rewards = []
        episode_actions = []
        job_queue = train_jobs.copy()
        step_count = 0
        max_steps = 1000
        
        while len(job_queue) > 0 and step_count < max_steps:
            if step_count % 50 == 0:
                print(f"Step {step_count}, Remaining jobs: {len(job_queue)}")
            
            state_tensor = torch.FloatTensor(state)
            action_probs = actor(state_tensor)
            dist = Categorical(action_probs)
            action = dist.sample()
            
            current_job = job_queue[0]
            next_state, reward, done, info = env.step(action.item(), current_job)
            
            if info['scheduled']:
                job_queue.pop(0)
            
            episode_rewards.append(reward)
            episode_actions.append(action.item())
            state = next_state
            step_count += 1
            
            if done:
                print(f"Episode finished after {step_count} steps")
                break
        
        # Print basic metrics before analytics update
        total_reward = sum(episode_rewards)
        schedule_rate = sum(1 for a in episode_actions if a == 0) / len(episode_actions)
        
        print(f"Basic metrics for episode {episode}:")
        print(f"Total Reward: {total_reward:.2f}")
        print(f"Schedule Rate: {schedule_rate:.2%}")
        print(f"Total Cost: ${env.total_cost:.2f}")
        
        # Update analytics with error handling
        try:
            metrics = analytics.update(env, episode)
            print(f"Peak Ratio: {metrics['peak_ratio']:.2%}")
            print(f"Cost Efficiency: {metrics['cost_efficiency']:.2%}")
        except Exception as e:
            print(f"Warning: Analytics update failed: {str(e)}")
            print("Continuing with training...")
        
        if total_reward > best_reward:
            best_reward = total_reward
            torch.save(actor.state_dict(), 'best_scheduler_actor.pth')
    
    return analytics

# Run with error handling
try:
    print("Starting experiment...")
    analytics = run_experiment()
    print("Experiment completed successfully")
except Exception as e:
    print(f"Error occurred: {str(e)}")
    import traceback
    traceback.print_exc()
    
    def plot_metrics(self):
        """Create visualization of metrics"""
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        # Plot 1: Peak vs Off-peak consumption
        episodes = [m['episode'] for m in self.episode_metrics]
        peak_ratios = [m['peak_ratio'] for m in self.episode_metrics]
        axes[0, 0].plot(episodes, peak_ratios)
        axes[0, 0].set_title('Peak Hour Consumption Ratio')
        axes[0, 0].set_ylabel('Peak/Total Ratio')
        axes[0, 0].axhline(y=0.4, color='g', linestyle='--', label='Target Ratio')
        axes[0, 0].legend()
        
        # Plot 2: Cost Efficiency
        cost_efficiency = [m['cost_efficiency'] for m in self.episode_metrics]
        axes[0, 1].plot(episodes, cost_efficiency)
        axes[0, 1].set_title('Cost Efficiency Over Time')
        axes[0, 1].set_ylabel('Efficiency Ratio')
        
        # Plot 3: Power Distribution
        avg_power = [m['avg_power'] for m in self.episode_metrics]
        high_power_ratio = [m['high_power_ratio'] for m in self.episode_metrics]
        axes[1, 0].plot(episodes, avg_power, label='Avg Power')
        axes[1, 0].plot(episodes, high_power_ratio, label='High Power Ratio')
        axes[1, 0].set_title('Power Consumption Patterns')
        axes[1, 0].legend()
        
        # Plot 4: Cost per kWh
        cost_per_kwh = [m['avg_cost_per_kwh'] for m in self.episode_metrics]
        axes[1, 1].plot(episodes, cost_per_kwh)
        axes[1, 1].set_title('Average Cost per kWh')
        axes[1, 1].set_ylabel('Cost ($)')
        
        plt.tight_layout()
        plt.show()

# Modify training loop to use analytics
def run_experiment():
    print("Initializing environment...")
    env = HPCEnvironment(
        gnn_model=model,
        feature_scaler=feature_scaler,
        power_scaler=power_scaler,
        num_nodes=100
    )
    
    print("Creating networks...")
    input_dims = 8
    n_actions = 2
    actor = ActorNetwork(input_dims, n_actions)
    critic = CriticNetwork(input_dims)
    
    print("Creating analytics tracker...")
    analytics = SchedulerAnalytics()
    
    print("Preparing job queue...")
    # Start with a smaller job queue for testing
    train_jobs = prepare_job_queue(gnn_train_data, n_jobs=100)
    print(f"Job queue size: {len(train_jobs)}")
    
    print("\nStarting training...")
    best_reward = float('-inf')
    
    for episode in range(100):
        print(f"\nStarting Episode {episode}")
        state = env.reset()
        episode_rewards = []
        episode_actions = []
        job_queue = train_jobs.copy()
        step_count = 0
        max_steps = 1000
        
        while len(job_queue) > 0 and step_count < max_steps:
            if step_count % 50 == 0:
                print(f"Step {step_count}, Remaining jobs: {len(job_queue)}")
            
            state_tensor = torch.FloatTensor(state)
            action_probs = actor(state_tensor)
            dist = Categorical(action_probs)
            action = dist.sample()
            
            current_job = job_queue[0]
            next_state, reward, done, info = env.step(action.item(), current_job)
            
            if info['scheduled']:
                job_queue.pop(0)
            
            episode_rewards.append(reward)
            episode_actions.append(action.item())
            state = next_state
            step_count += 1
            
            if done:
                print(f"Episode finished after {step_count} steps")
                break
        
        # Print basic metrics before analytics update
        total_reward = sum(episode_rewards)
        schedule_rate = sum(1 for a in episode_actions if a == 0) / len(episode_actions)
        
        print(f"Basic metrics for episode {episode}:")
        print(f"Total Reward: {total_reward:.2f}")
        print(f"Schedule Rate: {schedule_rate:.2%}")
        print(f"Total Cost: ${env.total_cost:.2f}")
        
        # Update analytics with error handling
        try:
            metrics = analytics.update(env, episode)
            print(f"Peak Ratio: {metrics['peak_ratio']:.2%}")
            print(f"Cost Efficiency: {metrics['cost_efficiency']:.2%}")
        except Exception as e:
            print(f"Warning: Analytics update failed: {str(e)}")
            print("Continuing with training...")
        
        if total_reward > best_reward:
            best_reward = total_reward
            torch.save(actor.state_dict(), 'best_scheduler_actor.pth')
    
    return analytics

# Run with error handling
try:
    print("Starting experiment...")
    analytics = run_experiment()
    print("Experiment completed successfully")
except Exception as e:
    print(f"Error occurred: {str(e)}")
    import traceback
    traceback.print_exc()


class BaselineScheduler:
    def __init__(self, num_nodes=100):
        self.num_nodes = num_nodes
        self.peak_hour_cost = 2.5
        self.off_peak_cost = 0.6
    
    def run_simulation(self, jobs):
        """Run FCFS scheduling simulation"""
        current_time = pd.Timestamp.now().replace(hour=0, minute=0)
        node_status = [True] * self.num_nodes
        running_jobs = []
        total_cost = 0
        peak_consumption = 0
        total_consumption = 0
        job_metrics = []
        
        # Process each job in order
        for job in jobs:
            # Wait until enough nodes are available
            while sum(node_status) < job['num_nodes_alloc']:
                # Find next job completion
                next_completion = min([end_time for _, _, end_time in running_jobs])
                current_time = next_completion
                
                # Update node status
                new_running = []
                for run_job, nodes, end_time in running_jobs:
                    if end_time > current_time:
                        new_running.append((run_job, nodes, end_time))
                    else:
                        for node in nodes:
                            node_status[node] = True
                running_jobs = new_running
            
            # Schedule job
            allocated_nodes = []
            for i, available in enumerate(node_status):
                if available and len(allocated_nodes) < job['num_nodes_alloc']:
                    allocated_nodes.append(i)
                    node_status[i] = False
            
            # Calculate power and cost
            power = job['mean_node_power']
            duration_hours = job['run_time'] / 3600
            is_peak = 8 <= current_time.hour < 20
            price = self.peak_hour_cost if is_peak else self.off_peak_cost
            
            energy_used = power * duration_hours
            cost = energy_used * price
            
            if is_peak:
                peak_consumption += energy_used
            total_consumption += energy_used
            total_cost += cost
            
            # Record job completion
            end_time = current_time + pd.Timedelta(seconds=job['run_time'])
            running_jobs.append((job, allocated_nodes, end_time))
            
            job_metrics.append({
                'job_id': job['job_id'],
                'start_time': current_time,
                'end_time': end_time,
                'is_peak': is_peak,
                'energy_used': energy_used,
                'cost': cost
            })
        
        return {
            'total_cost': total_cost,
            'total_consumption': total_consumption,
            'peak_consumption': peak_consumption,
            'peak_ratio': peak_consumption / total_consumption,
            'job_metrics': job_metrics
        }

def compare_schedulers():
    """Compare power-aware vs baseline scheduling"""
    # Prepare test jobs
    test_jobs = prepare_job_queue(gnn_test_data, n_jobs=100)
    
    # Run baseline scheduler
    print("Running baseline scheduler...")
    baseline = BaselineScheduler()
    baseline_results = baseline.run_simulation(test_jobs)
    
    # Run power-aware scheduler
    print("\nRunning power-aware scheduler...")
    env = HPCEnvironment(
        gnn_model=model,
        feature_scaler=feature_scaler,
        power_scaler=power_scaler,
        num_nodes=100
    )
    
    state = env.reset()
    power_aware_metrics = []
    job_queue = test_jobs.copy()
    
    while len(job_queue) > 0:
        state_tensor = torch.FloatTensor(state)
        action_probs = actor(state_tensor)
        dist = Categorical(action_probs)
        action = dist.sample()
        
        current_job = job_queue[0]
        next_state, reward, done, info = env.step(action.item(), current_job)
        
        if info['scheduled']:
            job_queue.pop(0)
            if 'energy_cost' in info:
                power_aware_metrics.append(info)
        
        state = next_state
        if done:
            break
    
    # Print comparison
    print("\nResults Comparison:")
    print("Baseline Scheduler:")
    print(f"Total Cost: ${baseline_results['total_cost']:.2f}")
    print(f"Total Energy: {baseline_results['total_consumption']:.2f} kWh")
    print(f"Peak Ratio: {baseline_results['peak_ratio']*100:.2f}%")
    
    print("\nPower-Aware Scheduler:")
    print(f"Total Cost: ${env.total_cost:.2f}")
    print(f"Total Energy: {env.energy_consumption:.2f} kWh")
    print(f"Peak Ratio: {(env.peak_consumption/env.energy_consumption)*100:.2f}%")
    
    # Calculate improvements
    cost_improvement = (baseline_results['total_cost'] - env.total_cost) / baseline_results['total_cost'] * 100
    peak_improvement = (baseline_results['peak_ratio'] - env.peak_consumption/env.energy_consumption) * 100
    
    print("\nImprovements:")
    print(f"Cost Reduction: {cost_improvement:.2f}%")
    print(f"Peak Ratio Reduction: {peak_improvement:.2f} percentage points")
    
    return baseline_results, env

# Run comparison
baseline_results, power_aware_env = compare_schedulers()


class BaselineScheduler:
    def __init__(self, num_nodes=100):
        self.num_nodes = num_nodes
        self.peak_hour_cost = 2.5
        self.off_peak_cost = 0.6
        
    def run_simulation(self, jobs):
        current_time = pd.Timestamp.now().replace(hour=0, minute=0)
        node_status = [True] * self.num_nodes
        running_jobs = []
        total_cost = 0
        peak_consumption = 0
        total_consumption = 0
        scheduled_jobs = []
        
        print(f"Starting simulation at {current_time}")
        
        for job_idx, job in enumerate(jobs):
            print(f"\nProcessing job {job_idx}")
            # Calculate power and time
            power = job['mean_node_power']
            duration_hours = job['run_time'] / 3600
            
            # Check if it's peak hours
            is_peak = 8 <= current_time.hour < 20
            price = self.peak_hour_cost if is_peak else self.off_peak_cost
            
            print(f"Time: {current_time.hour}:00, Peak hour: {is_peak}")
            
            energy_used = power * duration_hours
            cost = energy_used * price
            
            if is_peak:
                peak_consumption += energy_used
            total_consumption += energy_used
            total_cost += cost
            
            scheduled_jobs.append({
                'job_id': job_idx,
                'start_time': current_time,
                'is_peak': is_peak,
                'energy_used': energy_used,
                'cost': cost
            })
            
            # Move time forward
            current_time += pd.Timedelta(minutes=30)
        
        print("\nSimulation complete")
        return {
            'total_cost': total_cost,
            'total_consumption': total_consumption,
            'peak_consumption': peak_consumption,
            'peak_ratio': peak_consumption / (total_consumption + 1e-8),
            'scheduled_jobs': scheduled_jobs
        }

def run_power_aware_simulation(jobs, env, actor):
    print("\nRunning power-aware simulation...")
    state = env.reset()
    metrics = []
    job_queue = jobs.copy()
    current_time = pd.Timestamp.now().replace(hour=0, minute=0)
    
    while len(job_queue) > 0:
        current_job = job_queue[0]
        
        # Get current hour and peak status
        is_peak = 8 <= current_time.hour < 20
        print(f"\nTime: {current_time.hour}:00, Peak hour: {is_peak}")
        print(f"Job power: {current_job['mean_node_power']:.2f}, Duration: {current_job['run_time']/3600:.2f}h")
        
        # Get action from policy
        state_tensor = torch.FloatTensor(state)
        action_probs = actor(state_tensor)
        probs = action_probs.detach().numpy()
        print(f"Action probabilities - Schedule: {probs[0]:.2f}, Wait: {probs[1]:.2f}")
        
        dist = Categorical(action_probs)
        action = dist.sample()
        
        # Take action
        next_state, reward, done, info = env.step(action.item(), current_job)
        
        if info['scheduled']:
            print("Job scheduled")
            job_queue.pop(0)
            metrics.append(info)
        else:
            print("Job delayed")
        
        current_time += pd.Timedelta(minutes=30)
        state = next_state
        
        if done:
            break
    
    return env, metrics

def compare_schedulers(test_jobs, max_jobs=20):
    # Run baseline
    print(f"\nTesting with first {max_jobs} jobs...")
    test_subset = test_jobs[:max_jobs]
    
    print("\nRunning baseline scheduler...")
    baseline = BaselineScheduler()
    baseline_results = baseline.run_simulation(test_subset)
    
    # Run power-aware
    env = HPCEnvironment(
        gnn_model=model,
        feature_scaler=feature_scaler,
        power_scaler=power_scaler,
        num_nodes=100
    )
    
    power_aware_env, power_metrics = run_power_aware_simulation(test_subset, env, actor)
    
    # Print detailed comparison
    print("\nDetailed Results Comparison:")
    print("\nBaseline Scheduler:")
    print(f"Total Cost: ${baseline_results['total_cost']:.2f}")
    print(f"Total Energy: {baseline_results['total_consumption']:.2f} kWh")
    print(f"Peak Energy: {baseline_results['peak_consumption']:.2f} kWh")
    print(f"Peak Ratio: {baseline_results['peak_ratio']*100:.2f}%")
    
    print("\nPower-Aware Scheduler:")
    print(f"Total Cost: ${power_aware_env.total_cost:.2f}")
    print(f"Total Energy: {power_aware_env.energy_consumption:.2f} kWh")
    print(f"Peak Energy: {power_aware_env.peak_consumption:.2f} kWh")
    print(f"Peak Ratio: {(power_aware_env.peak_consumption/power_aware_env.energy_consumption)*100:.2f}%")
    
    # Calculate improvements
    cost_improvement = (baseline_results['total_cost'] - power_aware_env.total_cost) / baseline_results['total_cost'] * 100
    peak_improvement = (baseline_results['peak_ratio'] - power_aware_env.peak_consumption/power_aware_env.energy_consumption) * 100
    
    print("\nImprovements:")
    print(f"Cost Reduction: {cost_improvement:.2f}%")
    print(f"Peak Ratio Reduction: {peak_improvement:.2f} percentage points")
    
    return baseline_results, power_aware_env

# Run comparison
test_jobs = prepare_job_queue(gnn_test_data, n_jobs=100)
baseline_results, power_aware_env = compare_schedulers(test_jobs, max_jobs=20)

/tmp/ipykernel_1989129/1740890553.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file_path, parse_dates=time_columns)


KeyboardInterrupt: 